# Decision Tree Model

This Notebook will do two things: 

1. Import a CSV file and convert SMILES into fingerprints
2. Apply those fingerprints as input to train a Decision Tree Model

Most of the code was sourced from ChatGPT to kick-start the project. Modifications were applied. 

In [4]:
# Imports 
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D import Generate, Gobbi_Pharm2D
from rdkit import RDConfig
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

After importing all the necessary libraries, we upload the CSV file with all the data. This file holds 15,619 molecules in the form of SMILES (text). Two additional columns of data provide, the International Chemical Identifier (InChi) and the acitivity (float) of the molecule. 

## Data and Fingerprints

In [5]:
# Load the CSV file
file_path = "../Data/Mtb_published_regression_AC_Cleaned(in).csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Ensure the file has the necessary columns
required_columns = {"SMILES", "InChi", "Activity"}
if not required_columns.issubset(data.columns):
    raise ValueError(f"The input CSV must contain the following columns: {required_columns}")
data.head()

,SMILES,InChi,Activity
0,Cc1nc2ccc(N)cc2s1,InChI=1S/C8H8N2S/c1-5-10-7-3-2-6(9)4-8(7)11-5/...,6.22185
1,Nc1ccc2ncccc2c1,InChI=1S/C9H8N2/c10-8-3-4-9-7(6-8)2-1-5-11-9/h...,4.42091
2,CC(NC(=O)Nc1nnc(C(F)(F)F)s1)(C(F)(F)F)C(F)(F)F,"InChI=1S/C8H5F9N4OS/c1-5(7(12,13)14,8(15,16)17...",5.54683
3,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)c1,InChI=1S/C25H29I2NO3/c1-4-7-11-22-23(18-10-8-9...,5.49894
4,O=[N+]([O-])c1cc([N+](=O)[O-])c(-c2cc[nH]n2)s1,InChI=1S/C7H4N4O4S/c12-10(13)5-3-6(11(14)15)16...,5.50390


After uploading the data, we convert the SMILES into fingerprints, leveraging the RDKit library. 

In [6]:
# Predefine the feature factory for pharmacophore generation

# Code from https://www.rdkit.org/docs/GettingStartedInPython.html
fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

# Convert SMILES to pharmacophore fingerprints
def smiles_to_pharmacophore_fingerprint(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            # Generate 2D pharmacophore fingerprint
            fp = Generate.Gen2DFingerprint(mol, Gobbi_Pharm2D.factory)
            return np.array(fp)  # Convert to array for compatibility
        else:
            return None
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None

# Generate fingerprints
data['fingerprint'] = data['SMILES'].apply(smiles_to_pharmacophore_fingerprint)
num_molecules_raw = data["SMILES"].size

KeyboardInterrupt: 

After creating a set of fingerprints, we then remove any invalid molecules. 

In [35]:
# Remove rows with invalid SMILES
data = data.dropna(subset=['fingerprint'])
X = np.array(data['fingerprint'].tolist())
y = data['Activity'].values
print(f"Original Dataset has {num_molecules_raw} molecules")
num_fingerprints = data["fingerprint"].size
print(f"{num_molecules_raw - num_fingerprints} molecules were removed")

To check how many molecules were removed, we print the size of the original dataset with the new dataset

## Model

The model will split the dataset into 8:1:1 - training:test:validation. 

In [37]:
# Initial 80-20 split (train, test & validation)
hitch_hiker = 42
non_training_size = 0.2
validation_size = 0.5
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=non_training_size, random_state=hitch_hiker)

# Further split temp into 50-50 (test and validation)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=validation_size, random_state=hitch_hiker)

# Check the sizes
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")
print(f"Validation set size: {len(X_val)}")

# Build and train the decision tree model
model = DecisionTreeRegressor(random_state=hitch_hiker)
model.fit(X_train, y_train)

Training set size: 12494
Testing set size: 1562
Validation set size: 1562


DecisionTreeRegressor(random_state=42)

In [38]:
# Evaluate on test and validation sets
y_test_pred = model.predict(X_test)
y_val_pred = model.predict(X_val)

mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

print(f"Test Set - Mean Squared Error: {mse_test}, R^2 Score: {r2_test}")
print(f"Validation Set - Mean Squared Error: {mse_val}, R^2 Score: {r2_val}")

# Save the model (optional)
import joblib
joblib.dump(model, "decision_tree_model.pkl")

Test Set - Mean Squared Error: 0.5354836919958316, R^2 Score: 0.18800811118622807
Validation Set - Mean Squared Error: 0.5264068086123372, R^2 Score: 0.23035154878499686


['decision_tree_model.pkl']

From the MSE, we can see that the error is small, a good sign. However the Regression value is very small and doesnt show much clarification. Increasing radius to 3 reduces the R2 score, but increasing the nbits representation of the fingerprints increased it. 

## Visualise